In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# https://github.com/leondgarse/keras_efficientnet_v2
!pip install -U keras-efficientnet-v2

In [85]:
import keras_efficientnet_v2
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from sklearn.metrics import confusion_matrix, classification_report

In [86]:
batch_size = 64
imgSize = 240

# Construct a tf.data.Dataset
(train_ds, valid_ds) , ds_info = tfds.load('oxford_flowers102', split=['test', 'train'], shuffle_files=True, as_supervised=True, with_info=True)

NUM_CLASSES = ds_info.features["label"].num_classes
classArr = ds_info.features["label"].names

In [87]:
# data_augmentation = keras.Sequential(
#   [
#     tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#     tf.keras.layers.RandomRotation(0.1),
#     tf.keras.layers.RandomZoom(0.1),
#     #tf.keras.layers.RandomContrast(0.1),
#   ]
# )

trainDsTranform = keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.Resizing(imgSize, imgSize),
    tf.keras.layers.Rescaling(1./127.5, offset=-1)
  ]
)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgSize, imgSize),
  tf.keras.layers.Rescaling(1./127.5, offset=-1)
])

In [88]:
expendRound = 2
temp_ds = train_ds
for i in range(expendRound):
    train_ds = train_ds.concatenate(temp_ds)

#train_ds = train_ds.map(lambda image,label:(data_augmentation(image),label)).batch(32)
train_ds = train_ds.map(lambda image,label:(trainDsTranform(image),label)).batch(batch_size)


valid_ds = valid_ds.map(lambda image,label:(resize_and_rescale(image),label)).batch(batch_size)

In [ ]:
baseModel = keras_efficientnet_v2.EfficientNetV1B1(pretrained="noisy_student", num_classes=NUM_CLASSES, drop_connect_rate=0.3)

baseModel.trainable = True
print("Layers count", len(baseModel.layers))

fine_tune_at = int( len(baseModel.layers) * 0.50) 
for layer in baseModel.layers[:fine_tune_at]:
   layer.trainable = False
  
epochsRound = 14
base_learning_rate = 0.0001

checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

baseModel.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = baseModel.fit(train_ds, epochs=epochsRound, validation_data=valid_ds, callbacks=[model_checkpoint_callback], batch_size=batch_size)

test_loss, test_acc = baseModel.evaluate(valid_ds, verbose=2)
print(test_acc)

now = datetime.now()
current_time = now.strftime("%b-%d-%Y_%H:%M:%S")
srt = "/content/drive/MyDrive/savedModel/" + "efficientnetV1B0" + current_time + ".h5"

baseModel.load_weights(checkpoint_filepath)
baseModel.save(srt)

In [ ]:
y_pred = baseModel.predict(valid_ds)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = tf.concat([lab for img, lab in valid_ds], axis=0)
report = classification_report(predicted_categories, true_categories, target_names=classArr, zero_division=0, digits=4)

print(report)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochsRound)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# References  

1. Keras EfficientNetV2:  
https://github.com/leondgarse/keras_efficientnet_v2  
2. Transfer learning and fine-tuning  
https://www.tensorflow.org/tutorials/images/transfer_learning?hl=zh-tw  
3. Training a neural network on MNIST with Keras  
https://www.tensorflow.org/datasets/keras_example
